## Standard and Adversarial training SmallImageNet

In [2]:
import torch
from pytorch_trainers_interpretability.interpretability_eval import IntegratedGrad
from pytorch_trainers_interpretability.interpretability_eval.lime import LimeEval
import matplotlib.pyplot as plt
import numpy as np
from textwrap import wrap
from pytorch_trainers_interpretability.trainers import StandardTrainer, AdversarialTrainer
from pytorch_trainers_interpretability.interpretability_eval.integrated_grad import IntegratedGrad
from pytorch_trainers_interpretability.attack import Attacker, L2Step, LinfStep
from pytorch_trainers_interpretability.trainers.tools import get_mean_and_std, AccEvaluator
from pytorch_trainers_interpretability.models.resnet  import ResNet50
from torchvision import datasets, transforms
from PIL import Image
import json
import torchvision
import copy
import os
import torch.nn as nn

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1619V_hLgH3mhZSVCYuYO1G7y0088A1vq' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1619V_hLgH3mhZSVCYuYO1G7y0088A1vq" -O "smallimagenet.tar.gz" && rm -rf /tmp/cookies.txt
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1t71KG_u-X-LCAFJ94Kg0pqNBajumEEsu' -O "smallimagenet150_labels.json"

In [ ]:
!tar -xzf ./smallimagenet.tar.gz ./

In [ ]:
traindir = os.path.join("./smallimagenet", 'train')
valdir = os.path.join("./smallimagenet", 'test')
normalize = transforms.Normalize(mean=[0.4808, 0.4512, 0.4072],
                                     std=[0.2687, 0.2610, 0.2742])
trainset = datasets.ImageFolder(traindir)
transform_train =   transforms.Compose([
          transforms.Resize(140),
          transforms.RandomResizedCrop(128),
          transforms.RandomHorizontalFlip(),
          transforms.ToTensor(),
    ])
transform_test = transforms.Compose([
          transforms.Resize(140),
          transforms.CenterCrop(128),
          transforms.ToTensor(),
    ])
testset = datasets.ImageFolder(valdir, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=200,
                                         shuffle=True)  
f = open('./smallimagenet150_labels.json')

classes = json.load(f)

In [ ]:
model = ResNet50(num_classes=150)
training_kwargs = {
    "model": model,
    "pretrained": False,
    "batch_size": 200,
    "adv_step": L2Step,
    "adv_epsilon": 0.5,
    "adv_iter": 20,
    "adv_lr": 0.1,
    "lr": 0.0001,
    "epochs": 100,
    "optimizer": "Adam",
    "weight_decay": 5e-4,
    "lr_scheduler": "CosineAnnealingLR",
    "testset": testset,
    "trainset": trainset,
    "transforms_train": transform_train,
    "transforms_test": transform_test,
    "input_normalizer": normalize,
    "save_path": "./smimagenet_standard_model"
}
trainer = StandardTrainer(**training_kwargs)
trainer()
trainer.eval()
trainer.eval_adv()

In [ ]:
model2 = ResNet50(num_classes=150)
training_kwargs2 = {
    "model": model2,
    "pretrained": False,
    "batch_size": 200,
    "adv_step": L2Step,
    "adv_epsilon": 0.5,
    "adv_iter": 20,
    "adv_lr": 0.1,
    "lr": 0.0001,
    "epochs": 100,
    "optimizer": "Adam",
    "weight_decay": 5e-4,
    "lr_scheduler": "CosineAnnealingLR",
    "testset": testset,
    "trainset": trainset,
    "transforms_train": transform_train,
    "transforms_test": transform_test,
    "input_normalizer": normalize,
    "save_path": "./smimagenet_robust_l2_model"
}
trainer2 = AdversarialTrainer(**training_kwargs2)
trainer2()
trainer2.eval_nat()
trainer2.eval_adv()

In [ ]:
model3 = ResNet50(num_classes=150)
training_kwargs3 = {
    "model": model3,
    "pretrained": False,
    "batch_size": 200,
    "adv_step": LinfStep,
    "adv_epsilon": 4/255,
    "adv_iter": 20,
    "adv_lr": 0.01,
    "lr": 0.0001,
    "epochs": 100,
    "optimizer": "Adam",
    "weight_decay": 5e-4,
    "lr_scheduler": "CosineAnnealingLR",
    "testset": testset,
    "trainset": trainset,
    "transforms_train": transform_train,
    "transforms_test": transform_test,
    "input_normalizer": normalize,
    "save_path": "./smimagenet_robust_linf_model"
}
trainer3 = AdversarialTrainer(**training_kwargs3)
trainer3()
trainer3.eval_nat()
trainer3.eval_adv()